In [26]:
import requests
import lxml.html
import re
import time

In [ ]:
response = requests.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
root = lxml.html.fromstring(response.content)
# 모든 링크를 절대 URL로 변환
root.make_links_absolute(response.url)

# 목록에서 javascript 제거
for a in root.cssselect('.view_box .book_tit a'):
    url = a.get('href')
    print(url)

In [28]:
def main():
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()  
    response = session.get('https://www.innisfree.com/kr/ko/BestSellerList.do')
    urls = scrape_list_page(response)
    for url in urls:
        time.sleep(1) # 1초간 대기
        response = session.get(url)  # Session을 사용해 상세 페이지를 추출
        ebook = scrape_detail_page(response)  # 상세 페이지에서 상세 정보를 추출
        print(ebook)  # 상세 정보 출력

In [19]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    #절대경로로 변환
    root.make_links_absolute(response.url)
    for a in root.cssselect('a.pdtCont'):
        url = a.get('href')
        # yield 구문으로 제너레이터의 요소 반환
        yield url

In [30]:
def scrape_detail_page(response):
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'name': root.cssselect('.pdtName')[0].text_content(),
        'price': root.cssselect('dd.price span.num')[0].text_content(),
        'score': root.cssselect('span.reviewNum')[0].text_content(),
        'review': root.cssselect('div.review span.reviewLength span.num')[0].text_content(),
        'content': [normalize_spaces(p.text_content())
            for p in root.cssselect('div.pdtViewCont p.pdtTxt')
            if normalize_spaces(p.text_content()) != '']
        
    }
    return ebook

In [22]:
#연결된 공백을 하나의 공백으로 변경
def normalize_spaces(s):
    return re.sub(r'\s+', ' ', s).strip()

In [32]:
if __name__ == '__main__':
    main()

{'url': 'https://www.innisfree.com/kr/ko/ProductView.do?prdSeq=28250', 'name': '블랙티 유스 인핸싱 앰플 30mL', 'price': '35,000', 'score': '4.8', 'review': '3549', 'content': ['지친 피부의 컨디션을 빠르게 회복시켜 주어 숙면한 듯 건강한 피부로 만들어 주는 나이트 집중 케어 앰플']}
{'url': 'https://www.innisfree.com/kr/ko/ProductView.do?prdSeq=16958', 'name': '비자 시카 밤 [대용량] 80mL', 'price': '32,000', 'score': '4.7', 'review': '1039', 'content': ['피부 스트레스 진정 및 피부 고민 부위 케어에 도움을 주는 젤 타입 밤 [대용량]']}
{'url': 'https://www.innisfree.com/kr/ko/ProductView.do?prdSeq=25416', 'name': '한란 럭키 박스 ', 'price': '12,500', 'score': '4.7', 'review': '552', 'content': ['눈 속에서 피어나는 제주한란의 생명력, 한란 인리치드 크림에 대한 사랑에 보답하는 마음으로 스페셜한 행운을 담은 한정판 [한란 럭키박스]']}
{'url': 'https://www.innisfree.com/kr/ko/ProductView.do?prdSeq=22596', 'name': '포레스트 포맨 올인원 에센스 4종 100mL', 'price': '25,000', 'score': '4.7', 'review': '945', 'content': ['숲에서 찾은 피부보호막 성분 블랙이스트를 함유하고, 남성 피부 고민을 케어해주어 건강하고 자신감 넘치는 피부로 가꾸어주는 올인원 에센스 4종']}
{'url': 'https://www.innisfree.com/kr/ko/ProductView.do?prdSeq=12

IndexError: list index out of range